## Wareki

The Wareki is a Japanese date format which includes an extra character to signify the era. The eras indicate the emperor at the time and they impact the year only. For example, H04/01/31 represents the 4th year of Heisei era, but the 01/31 is in fact January 31st of that year.

Modern Japan, which is considered the period beginning in 1868, is currently in its 5th era. This week's challenge is to create a macro that converts Wareki dates into Alteryx Datetime or vice versa. The macro should include a configuration option for Wareki to Date or Date to Wareki.

**Example:**

S550730 is 30 July 1980.

_Original challenge: https://community.alteryx.com/t5/Weekly-Challenge/Challenge-237-Wareki/td-p/648486_

In [173]:
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [174]:
df = pd.read_csv('.\challenge_237_input.csv', parse_dates = ['Start', 'End'])
df

,Era,Name,Start,End
0,M,Meiji,1868-10-23,1912-07-30
1,T,Taisho,1912-07-31,1926-12-25
2,S,Showa,1926-12-26,1989-01-07
3,H,Heisei,1989-01-08,2019-04-30
4,R,Reiwa,2019-05-01,NaT


## Wareki to date - if statement approach

In [176]:
def wareki_to_date(wareki):
    
    """
    Converts a Wareki date to a standard time format.
    
    Args:
        wareki: Wareki time as string in the following format: S550730
    """
    
    wareki_era = wareki[0]
    wareki_year = int(wareki[1:3])
    wareki_month = int(wareki[3:5])
    wareki_day = int(wareki[5:7])
    
    if wareki_era == 'M':
        era_year = (df.iloc[0, 2] + relativedelta(years = wareki_year - 1)).year
    elif wareki_era == 'T':
        era_year = (df.iloc[1, 2] + relativedelta(years = wareki_year - 1)).year
    elif wareki_era == 'S':
        era_year = (df.iloc[2, 2] + relativedelta(years = wareki_year - 1)).year
    elif wareki_era == 'H':
        era_year = (df.iloc[3, 2] + relativedelta(years = wareki_year - 1)).year 
    elif wareki_era == 'R':
        era_year = (df.iloc[4, 2] + relativedelta(years = wareki_year - 1)).year 
        
    return print(datetime(era_year, wareki_month, wareki_day))

In [196]:
wareki_to_date('M450522')

1912-05-22 00:00:00


## Wareki to date - nested function approach
This approach avoids writing out a long if statement. 

In [180]:
def wareki_to_date2(wareki):
    
    """
    Converts a Wareki date to a standard time format.
    
    Args:
        wareki: Wareki time as string in the following format: S550730
    """
    
    wareki_era = wareki[0]
    wareki_year = int(wareki[1:3])
    wareki_month = int(wareki[3:5])
    wareki_day = int(wareki[5:7])
    era_year = 0
    
    def start_date(era, row):
        """
        Returns the cell containing the start date of each era

        Args:
            era: the era, eg 'M', 'T' etc
            row: the row in which the era can be found
        """
        if wareki_era == era:
            nonlocal era_year
            era_year =  (df.iloc[row, 2] + relativedelta(years = wareki_year - 1)).year
            

    start_date('M', 0)
    start_date('T', 1)
    start_date('S', 2)
    start_date('H', 3)
    start_date('R', 4)
    
    return print(datetime(era_year, wareki_month, wareki_day))

In [197]:
wareki_to_date2('M450522')

1912-05-22 00:00:00


## Date to Wareki - if statement approach

In [212]:
def date_to_wareki(my_date):
    """
    Converts a date to Wareki format.
    
    Args:
        my_date: Standard date format as string (eg 1982-12-03)
    """
    
    date_format = datetime.strptime(my_date, '%Y-%m-%d')
    date_year = date_format.year
    date_month = date_format.month
    date_day = date_format.day
    
    # Add leading zeros to months and days
    if len(str(date_month)) == 1:
        date_month = '0' + str(date_month)
    
    if len(str(date_day)) == 1:
        date_day = '0' + str(date_day) 
    
    # Add era abbreviation and calculate years since beginning of era
    if date_format >= df.iloc[0, 2] and date_format <= df.iloc[0, 3]:
        wareki = 'M'
        new_date_year = date_year - df.iloc[0, 2].year + 1
    elif date_format >= df.iloc[1, 2] and date_format <= df.iloc[1, 3]:
        wareki = 'T'
        new_date_year = date_year - df.iloc[1, 2].year + 1
    elif date_format >= df.iloc[2, 2] and date_format <= df.iloc[2, 3]:
        wareki = 'S'
        new_date_year = date_year - df.iloc[2, 2].year + 1
    elif date_format >= df.iloc[3, 2] and date_format <= df.iloc[3, 3]:
        wareki = 'H'
        new_date_year = date_year - df.iloc[3, 2].year + 1
    elif date_format >= df.iloc[4, 2] and date_format <= df.iloc[4, 3]:
        wareki = 'R'
        new_date_year = date_year - df.iloc[4, 2].year + 1
        
    return wareki + str(new_date_year) + str(date_month) + str(date_day)

In [214]:
date_to_wareki('1999-03-21')

'H110321'